In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [55]:
raw = pd.read_csv('data/Iowa_Liquor_Sales.csv',dtype={'Zip Code': object, 'Item Number': object}, parse_dates=['Date'])

In [56]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [57]:
# # Item Number and Zip Code to string
# # Date to datetime
# raw['Zip Code'] = raw['Zip Code'].apply(str)
# raw['Item Number'] = raw['Item Number'].apply(str)


# Product

In [58]:
# Product df
product = raw[['Item Number', 'Item Description', 'Category', 'Category Name', 'Pack', 'Bottle Volume (ml)']]
product['Category Name'] = product['Category Name'].str.lower()
product['Item Description'] = product['Item Description'].str.lower()

<ipython-input-58-085711850bfe>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product['Category Name'] = product['Category Name'].str.lower()
<ipython-input-58-085711850bfe>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product['Item Description'] = product['Item Description'].str.lower()


In [59]:
# Clean up Item Number to Category ID mapping 

In [60]:
# Item being mapped to multiple categories?  -- use the majority category 

prod_cat = product.groupby(['Item Number']).agg({'Category': 'value_counts'}).rename(columns = {'Category': 'Category Count'})
prod_cat = prod_cat.reset_index()
prod_cat['Rank'] = prod_cat.groupby('Item Number')['Category Count'].rank(ascending = False)
prod_cat_unique = prod_cat[prod_cat['Rank']==1]

In [61]:
product = pd.merge(product, prod_cat_unique, how = 'left', on = 'Item Number')
product.drop(['Category_x', 'Category Count', 'Rank'], axis=1, inplace=True)
product.rename(columns = {'Category_y': 'Category'}, inplace=True)

In [62]:
# Clean up category name so that category ID and category name has unique mapping 
cat_group = product.groupby(['Category', 'Category Name'])['Item Number'].count().reset_index().rename(columns = {'Item Number': 'count'})
cat_group['rank'] = cat_group.groupby('Category')['count'].rank(ascending = False)
cat_group_unique = cat_group[cat_group['rank'] == 1]

In [63]:
# Check 1031200.0
product = pd.merge(product, cat_group_unique, how = 'left', on = 'Category')
product.drop(['Category Name_x', 'count', 'rank'], axis=1, inplace = True)
product.rename(columns = {'Category Name_y': 'Category Name'}, inplace=True )

In [64]:
# Dealing with inconsistent product names - using the majority name 
prod_name = product.groupby('Item Number').agg({'Item Description': 'value_counts'})
prod_name.rename(columns = {'Item Description': 'name count'}, inplace=True)
prod_name.reset_index(inplace=True)
prod_name['rank'] = prod_name.groupby('Item Number')['name count'].rank(ascending = False)
prod_name_unique = prod_name[prod_name['rank']==1]

In [65]:
product = pd.merge(product, prod_name_unique, how = 'left', on = 'Item Number')
product.drop(['Item Description_x', 'name count', 'rank'], axis=1, inplace=True)
product.rename(columns = {'Item Description_y': 'Item Description'}, inplace=True)

In [66]:
# How to deal with inconsistent pack and bottle volume? 
# - Drop pack, not useful for our analysis 
# Keep the majority bottle volume 

In [67]:
bottle_vol = product.groupby('Item Number').agg({'Bottle Volume (ml)': 'value_counts'}).rename(columns = {'Bottle Volume (ml)': 'count'}).reset_index()
bottle_vol['rank'] = bottle_vol.groupby('Item Number')['count'].rank(ascending = False)
bottle_vol_unique = bottle_vol[bottle_vol['rank'] == 1]

In [68]:
product = pd.merge(product, bottle_vol_unique, how = 'left', on = 'Item Number')
# product.drop(['Bottle Volume (ml)_x', 'count', 'rank'], axis=1, inplace= True)
product.rename(columns = {'Bottle Volume (ml)_y': 'Bottle Volume (ml)'}, inplace= True)
product.drop(['Bottle Volume (ml)_x', 'count', 'rank'], axis=1, inplace = True)
product.drop(['Pack'], axis=1, inplace=True)
product_clean = product.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=True)

In [69]:
product_clean.head()

,Item Number,Category,Category Name,Item Description,Bottle Volume (ml)
0,36978,1031080.0,vodka 80 proof,nikolai vodka,1750.0
1,24453,1011100.0,blended whiskies,kessler blend whiskey,200.0
2,41989,1031200.0,vodka flavored,uv cake vodka,750.0
3,15248,1012100.0,canadian whiskies,windsor canadian pet,1750.0
4,43410,1062300.0,flavored rum,captain morgan parrot bay coconut,750.0


In [70]:
# proof
proof = pd.read_csv('data/Iowa_Liquor_Products.csv')
proof = proof[['Item Number', 'Proof']]
proof['Item Number'] =proof['Item Number'].apply(str) 
product_clean = pd.merge(product_clean, proof, how = 'left', on = 'Item Number')
#product_clean.to_csv("product_clean.csv")

In [71]:
# add category new
data = product_clean

In [72]:
data['Item Description'] = data['Item Description'].apply(str)
data['Category Name'] = data['Category Name'].apply(str)

In [73]:
data['category_new'] = np.where(data['Category Name'].str.contains('vodka'),'vodka',
                       np.where(data['Category Name'].str.contains('whisk|bourbon|scotch'),'whisky', 
                        np.where(data['Category Name'].str.contains('rum'), 'rum',
                        np.where(data['Category Name'].str.contains('cocktail'), 'cocktails',
                        np.where(data['Category Name'].str.contains('tequila|mezcal'), 'tequila',
                        np.where(data['Category Name'].str.contains('schnapps'), 'schnapps',
                        np.where(data['Category Name'].str.contains('gins'), 'gin',
                        np.where(data['Category Name'].str.contains('brandies'), 'brandy',
                        np.where(data['Category Name'].str.contains('liqueur|triple|creme|amaretto|anisette'), 'liqueur', 'other')))))))))

In [74]:
data[data.category_new == 'distilled spirits']['Category Name'].unique()

array([], dtype=object)

In [75]:
data[data.category_new == 'distilled spirits']['Item Description'].unique()

array([], dtype=object)

In [76]:
data.loc[data['category_new']=='other', 'category_new'] = np.where(data.loc[data['category_new']=='other']['Item Description'].str.contains('vodka'),'vodka',
                       np.where(data.loc[data['category_new']=='other']['Item Description'].str.contains('whisk|bourbon|scotch'),'whisky', 
                        np.where(data.loc[data['category_new']=='other']['Item Description'].str.contains('rum'), 'rum',
                        np.where(data.loc[data['category_new']=='other']['Item Description'].str.contains('cocktail'), 'cocktails',
                        np.where(data.loc[data['category_new']=='other']['Item Description'].str.contains('tequila|mezcal'), 'tequila',
                        np.where(data.loc[data['category_new']=='other']['Item Description'].str.contains('schnapps'), 'schnapps',
                        np.where(data.loc[data['category_new']=='other']['Item Description'].str.contains('gins'), 'gin',
                        np.where(data.loc[data['category_new']=='other']['Item Description'].str.contains('brandies'), 'brandy',
                        np.where(data.loc[data['category_new']=='other']['Item Description'].str.contains('liqueur|triple|creme|amaretto|anisette'), 'liqueur', 'other')))))))))


In [79]:
data.head()

,Item Number,Category,Category Name,Item Description,Bottle Volume (ml),Proof,category_new,imported
0,36978,1031080.0,vodka 80 proof,nikolai vodka,1750.0,80.0,vodka,domestic
1,24453,1011100.0,blended whiskies,kessler blend whiskey,200.0,80.0,whisky,domestic
2,41989,1031200.0,vodka flavored,uv cake vodka,750.0,60.0,vodka,domestic
3,15248,1012100.0,canadian whiskies,windsor canadian pet,1750.0,80.0,whisky,domestic
4,43410,1062300.0,flavored rum,captain morgan parrot bay coconut,750.0,42.0,rum,domestic


In [78]:
data['imported'] = np.where(data['Category Name'].str.contains('imported'), 'imported', 'domestic')

In [80]:
data.rename(columns = {'Category': 'category_id', 'Category Name': 'category_name',
                      'Item Description': 'product_name', 'Bottle Volume (ml)': 'bottle_vol_ml',
                      'Proof': 'proof'}, inplace=True)

In [81]:
product_clean = data
product_clean.to_csv("product_clean.csv",index_label=False)

# Price 

In [37]:
price = raw[['Item Number', 'Date', 'State Bottle Retail', 'State Bottle Cost']]
# price.groupby(['Item Number', 'Date']).head(1)
product_price = price.groupby(['Item Number', 'Date']).agg({'State Bottle Retail':'mean','State Bottle Cost':'mean'})
product_price.reset_index(inplace=True)


In [38]:
product_price.head()

,Item Number,Date,State Bottle Retail,State Bottle Cost
0,100001,2016-10-03,12.0,8.0
1,100001,2016-10-04,12.0,8.0
2,100001,2016-10-05,12.0,8.0
3,100001,2016-10-06,12.0,8.0
4,100001,2016-10-07,12.0,8.0


In [39]:
product_price.to_csv("product_price_clean.csv",index_label=False)

# Vendors

In [40]:
# Clean Vendors Name
Vendors = raw[['Vendor Number','Vendor Name']]


In [41]:
Vendors_group = Vendors.groupby('Vendor Number').agg({'Vendor Name': 'value_counts'})
Vendors_group.rename({'Vendor Name':'Vendor Count'},axis=1,inplace = True)
Vendors_group = Vendors_group.reset_index()
Vendors_group['rank'] = Vendors_group.groupby('Vendor Number')['Vendor Count'].rank(ascending = False)

In [42]:
Vendors_final = Vendors_group[Vendors_group['rank'] == 1.0].drop(['Vendor Count','rank'],axis=1)

In [43]:
Vendors_final.head()

,Vendor Number,Vendor Name
0,10.0,"A Hardy / U.S.A., Ltd."
2,14.0,"Adamba Imports Int'l, Inc."
3,27.0,Anheuser-Busch/Longtail Libations
4,33.0,Bmc Imports
6,35.0,"Bacardi U.S.A., Inc."


In [45]:
Vendors_final.to_csv('Vendors.csv',index_label=False)

# Transactions

In [44]:
Transcations = raw[['Invoice/Item Number','Store Number','Vendor Number','Item Number','Bottles Sold','Volume Sold (Liters)',
                    'Sale (Dollars)','Date']]

In [45]:
Transcations.columns = ['Transaction ID', 'Store Number', 'Vendor Number', 'Item Number',
       'Bottles Sold', 'Volume Sold', 'Sale', 'Date']

In [54]:
Transcations.to_csv('Transactions.csv',index_label=False)

# Stores

In [48]:
# Store df
store = raw.loc[:,['Store Number', 'Store Name', 'Address', 'City', 'Zip Code', 'Store Location', 'County Number', 'County']]
# lower case name
store.loc[:,'Store Name'] = store['Store Name'].str.lower()
store.loc[:,'Address'] = store['Address'].str.lower()
store.loc[:,'City'] = store['City'].str.lower()
store.loc[:,'County'] = store['County'].str.lower()

# replace null with string so the NaN is not contagious
store.replace(np.nan, 'missing', inplace = True)

##### Deal with varying store names

# Groupby store number and value count to get number of times store name matches to store number
store_name = store.groupby(['Store Number']).agg({'Store Name': 
                                                  'value_counts'}).rename(columns = {'Store Name': 
                                                                                     'Name Count'}).reset_index()

# rank the store names within each store number
store_name['Rank'] = store_name.groupby('Store Number')['Name Count'].rank(ascending = False)

# pick most frequent name
store_name_unique = store_name.loc[store_name.Rank == 1].drop(['Name Count', 
                                                               'Rank'], axis=1)

# drop columns from store
store = store.drop(['Store Name'], axis=1)

# join the ranked data to store df
store = pd.merge(store, store_name_unique, how='left', 
                 on='Store Number')

#### Deal with varying locations for each store number

# Business could have moved over the years
store_loc = store.groupby(['Store Number', 
                           'County', 'County Number', 'City', 
                           'Zip Code', 'Store Location']).agg({'Address': 
                                                               'value_counts'}).rename(columns = {'Address': 
                                                                                     'Address Count'}).reset_index()

# rank all location info within each store number
store_loc['Rank'] = store_loc.groupby('Store Number')['Address Count'].rank(ascending = False)

# pick most frequent name
store_loc_unique = store_loc.loc[store_loc.Rank == 1].drop(['Address Count', 
                                                            'Rank'], axis=1)

# drop columns from store
store = store.drop(['Address','City', 'Zip Code', 'Store Location', 
                    'County Number', 'County'], axis=1)

# join the ranked data to store df
store = pd.merge(store, store_loc_unique, how='left', 
                 on='Store Number')

# Drop duplicates
store = store.drop_duplicates()

## Adding Store Types

# Add column of zeros
store['Store Type'] = np.zeros(store.shape[0])

#### Categorize stores with string contains 
#- order matters here

store['Store Type'].loc[store['Store Name'].str.contains(
    "food|market|super valu|saver|groc")] = 'Other Grocery or Convenience'

store['Store Type'].loc[store['Store Name'].str.contains(
    "econ-o-mart|mart|quik|pit stop|quick|kwik|general store|convenience|gas|circle k|petro|stop|casey's|country store|yesway|kum|7-eleven|station|express|fill r up|fuel|new star| go |the boonedocks")] = 'Convenience Store'

store['Store Type'].loc[store['Store Name'].str.contains(
    "hy-vee|wal-mart|fareway store|super mar|big g|sac city food pride|sam's club|supermarket|shop n save|grocery|target|dahl's|costco|whole foods|jeff's|hometown|mepo")] = 'Supermarket'

store['Store Type'].loc[store['Store Name'].str.contains(
    "liquor|spirits|tobacco|beverage|smoke|bottle|distil|wine|bootleg|northside one stop|beer|cigar|distrib|booze|brew|snuff|sauce")] = 'Liquor Tobacco Store'

store['Store Type'].loc[store['Store Name'].str.contains(
    "walgreens|cvs|drug")] = 'Drug Store'

store['Store Type'].loc[store['Store Name'].str.contains(
    "casino")] = 'Casino'

store['Store Type'].loc[store['Store Type']==0] = 'Other'

D:\anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [50]:
store.to_csv('stores.csv',index_label=False)

# Sqlite

## Create liquor database

In [4]:
# product_clean = pd.read_csv('product_clean.csv')
# product_price = pd.read_csv('product_price_clean.csv')
# Vendors_final = pd.read_csv('Vendors.csv')
# Transcations = pd.read_csv('Transactions.csv')
#store = pd.read_csv('stores.csv')

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Insert store table

In [52]:
import sqlite3
# connect to sqlite
conn = sqlite3.connect('liquor.db')
# make a cursor
cursor = conn.cursor()
# make tables
sql_query = '''CREATE TABLE stores(
                   'Store Number' INTEGER PRIMARY KEY,
                   'Store Name' varchar(250),
                   'County' varchar(250),
                   'County Number' varchar(250),
                   'City' varchar(250),
                   'Zip Code' varchar(250),
                   'Store Location' varchar(250),
                   'Address' varchar(250),
                   'Store Type' varchar(250));'''
# sql_query = 'DROP TABLE stores;'
cursor.execute(sql_query)
# insert data
store.to_sql('stores',conn,if_exists='append',index=False)

cursor.close()
conn.close()

D:\anaconda\lib\site-packages\pandas\core\generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [82]:
product_clean.columns

Index(['Item Number', 'category_id', 'category_name', 'product_name',
       'bottle_vol_ml', 'proof', 'category_new', 'imported'],
      dtype='object')

In [83]:
product_clean.head()

,Item Number,category_id,category_name,product_name,bottle_vol_ml,proof,category_new,imported
0,36978,1031080.0,vodka 80 proof,nikolai vodka,1750.0,80.0,vodka,domestic
1,24453,1011100.0,blended whiskies,kessler blend whiskey,200.0,80.0,whisky,domestic
2,41989,1031200.0,vodka flavored,uv cake vodka,750.0,60.0,vodka,domestic
3,15248,1012100.0,canadian whiskies,windsor canadian pet,1750.0,80.0,whisky,domestic
4,43410,1062300.0,flavored rum,captain morgan parrot bay coconut,750.0,42.0,rum,domestic


### Insert products table

In [86]:
import sqlite3
# connect to sqlite
conn = sqlite3.connect('liquor.db')
# make a cursor
cursor = conn.cursor()
# make tables
sql_query = '''CREATE TABLE products(
                   'Item Number' varchar(50) PRIMARY KEY,
                   'category_id' float(64),
                   'category_name' varchar(50),
                   'product_name' varchar(50),
                   'bottle_vol_ml' float(64),
                   'proof' float(64),
                   'category_new' varchar(50),
                   'imported' varchar(50)
                   );'''
#sql_query = 'DROP TABLE products;'
cursor.execute(sql_query)
# insert data
product_clean.to_sql('products',conn,if_exists='append',index=False)

cursor.close()
conn.close()

D:\anaconda\lib\site-packages\pandas\core\generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


### Insert product price table

In [87]:
# connect to sqlite
conn = sqlite3.connect('liquor.db')
# make a cursor
cursor = conn.cursor()
# make tables
sql_query = '''CREATE TABLE product_price(
                   'Item Number' varchar(50),
                   'Date' datetime64[ns],
                   'State Bottle Retail' float(64),
                   'State Bottle Cost' float(64),
                   primary key ('Item Number', 'Date'),
                   FOREIGN KEY ('Item Number') REFERENCES products('Item Number')
                   );
'''
# sql_query = 'DROP TABLE product_price;'
cursor.execute(sql_query)
# insert data
product_price.to_sql('product_price',conn,if_exists='append',index=False)

cursor.close()
conn.close()

### Insert Vendors table

In [88]:
# connect to sqlite
conn = sqlite3.connect('liquor.db')
# make a cursor
cursor = conn.cursor()
# make tables
sql_query = '''CREATE TABLE vendors(
                   'Vendor number' varchar(50) PRIMARY KEY,
                   'Vendor name' varchar(50));'''
#sql_query = 'DROP TABLE vendors;'
cursor.execute(sql_query)
# insert data
Vendors_final.to_sql('vendors',conn,if_exists='append',index=False)

cursor.close()
conn.close()

### Insert Transcation tables

In [89]:
# connect to sqlite
conn = sqlite3.connect('liquor.db')
# make a cursor
cursor = conn.cursor()
# make tables
sql_query = '''CREATE TABLE transactions(
                   'Transaction ID' varchar(50) PRIMARY KEY,
                   'Store Number' varchar(50),
                   'Vendor Number' varchar(50),
                   'Item Number' varchar(50),
                   'Bottles Sold' interger,
                   'Volume Sold' float(64),
                   'Sale' float(64),
                   'Date' datetime64[ns],
                   FOREIGN KEY ('Vendor Number') REFERENCES vendors('Vendor Numver'),
                   FOREIGN KEY ('Item Number','Date') REFERENCES product_price('Item Number','Date'),
                   FOREIGN KEY ('Store Number') REFERENCES stores('Store Number')
                   );
'''
#sql_query = 'DROP TABLE transactions;'
cursor.execute(sql_query)
# insert data
Transcations.to_sql('transactions',conn,if_exists='append',index=False)

cursor.close()
conn.close()

### Fetching table

In [92]:
# Try fetching products table 

# connect to sqlite
conn = sqlite3.connect('liquor.db')
# make a cursor
cursor = conn.cursor()
# select data and transform it to df
#query = '''SELECT "Vendor name" FROM vendors where "Vendor number"=10;'''
query = '''SELECT * FROM transactions  ;


'''
cursor.execute(query)
result = cursor.fetchall()

cursor.close()
conn.close()

In [93]:
len(result)

19445831

In [14]:
pd.DataFrame(result,columns=['Item Number', 'Category', 'Category Name', 'Item Description',
       'Bottle Volume (ml)', 'Proof'])

ValueError: 6 columns passed, passed data had 8 columns

In [ ]:
product_price.columns

In [ ]:
# Try fetching product_price table 

# connect to sqlite
conn = sqlite3.connect('liquor.db')
# make a cursor
cursor = conn.cursor()
# select data and transform it to df
#query = '''SELECT "Vendor name" FROM vendors where "Vendor number"=10;'''
query = '''SELECT * FROM product_price;


'''
cursor.execute(query)
result1 = cursor.fetchone()

cursor.close()
conn.close()

In [ ]:
result1